In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from datetime import datetime, timedelta
import random

spark = SparkSession.builder.appName("CreditUnionDashboard").getOrCreate()

# Set parameters
num_agents = 25
num_members = 300
num_tickets = 500
random.seed(42)

# Generate agents DataFrame
agents_data = []
start_date = datetime.strptime("2019-01-01", "%Y-%m-%d")
for i in range(num_agents):
    agents_data.append((
        f"A{100 + i}",
        f"Agent_{i}",
        random.choice(["East", "West", "Central"]),
        random.choice(["New York", "Texas", "California", "Florida"]),
        (start_date + timedelta(days=90 * i)).strftime("%Y-%m-%d")
    ))

agents_df = spark.createDataFrame(agents_data, ["agent_id", "name", "team", "location", "hire_date"])
agents_df.createOrReplaceTempView("agents")

# Generate members DataFrame
members_data = []
start_date = datetime.strptime("2022-01-01", "%Y-%m-%d")
for i in range(num_members):
    members_data.append((
        f"M{1000 + i}",
        random.randint(18, 70),
        random.choice(["Male", "Female", "Non-binary"]),
        random.choice(["CA", "NY", "TX", "FL", "IL", "WA"]),
        (start_date + timedelta(days=3 * i)).strftime("%Y-%m-%d")
    ))

members_df = spark.createDataFrame(members_data, ["member_id", "age", "gender", "state", "join_date"])
members_df.createOrReplaceTempView("members")

# Generate support tickets DataFrame
tickets_data = []
start_date = datetime.strptime("2024-01-01", "%Y-%m-%d")
for i in range(num_tickets):
    created_at = start_date + timedelta(days=random.randint(0, 180), hours=random.randint(0, 23), minutes=random.randint(0, 59))
    resolution_minutes = random.randint(5, 240)
    closed_at = created_at + timedelta(minutes=resolution_minutes)
    tickets_data.append((
        f"T{1000 + i}",
        created_at,
        closed_at,
        random.choice(["Phone", "Email", "Chat"]),
        random.choice([f"A{100 + j}" for j in range(num_agents)]),
        random.choice([f"M{1000 + j}" for j in range(num_members)]),
        random.choice(["Low", "Medium", "High"]),
        "Closed"
    ))

tickets_df = spark.createDataFrame(tickets_data, ["ticket_id", "created_at", "closed_at", "channel", "agent_id", "member_id", "priority", "status"])
tickets_df.createOrReplaceTempView("support_tickets")

# Generate survey responses DataFrame
survey_data = []
for row in tickets_df.collect():
    if random.random() < 0.8:
        survey_data.append((
            row['ticket_id'],
            row['member_id'],
            random.randint(1, 5),
            random.choice(["Very satisfied", "Helpful", "Quick resolution", "Could be better", "Unsatisfied", "Excellent support"])
        ))

surveys_df = spark.createDataFrame(survey_data, ["ticket_id", "member_id", "csat_rating", "comments"])
surveys_df.createOrReplaceTempView("survey_responses")

print(" Synthetic data created and loaded into views: agents, members, support_tickets, and survey_responses")


 Synthetic data created and loaded into views: agents, members, support_tickets, and survey_responses


In [0]:
from pyspark.sql.functions import col, unix_timestamp, when

# Add resolution_time column (in minutes)
tickets_enriched_df = spark.sql("SELECT * FROM support_tickets") \
    .withColumn("resolution_time_min", 
                (unix_timestamp("closed_at") - unix_timestamp("created_at")) / 60)

# Flag SLA compliance (1 if resolved in <= 60 mins, else 0)
tickets_enriched_df = tickets_enriched_df.withColumn(
    "sla_met", when(col("resolution_time_min") <= 60, 1).otherwise(0)
)

# Join with agents
joined_df = tickets_enriched_df.join(spark.table("agents"), on="agent_id", how="left")

# Join with members
joined_df = joined_df.join(spark.table("members"), on="member_id", how="left")

# Join with surveys
final_df = joined_df.join(spark.table("survey_responses"), on=["ticket_id", "member_id"], how="left")

# Register as view for SQL access or export
final_df.createOrReplaceTempView("dashboard_data")

# Show schema to confirm
final_df.printSchema()

# Preview sample
display(final_df.limit(10))


root
 |-- ticket_id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- agent_id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- closed_at: timestamp (nullable = true)
 |-- channel: string (nullable = true)
 |-- priority: string (nullable = true)
 |-- status: string (nullable = true)
 |-- resolution_time_min: double (nullable = true)
 |-- sla_met: integer (nullable = false)
 |-- name: string (nullable = true)
 |-- team: string (nullable = true)
 |-- location: string (nullable = true)
 |-- hire_date: string (nullable = true)
 |-- age: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- state: string (nullable = true)
 |-- join_date: string (nullable = true)
 |-- csat_rating: long (nullable = true)
 |-- comments: string (nullable = true)



ticket_id,member_id,agent_id,created_at,closed_at,channel,priority,status,resolution_time_min,sla_met,name,team,location,hire_date,age,gender,state,join_date,csat_rating,comments
T1000,M1274,A103,2024-02-29T03:46:00.000+0000,2024-02-29T04:40:00.000+0000,Email,High,Closed,54.0,1,Agent_3,East,New York,2019-09-28,35,Non-binary,IL,2024-04-02,1,Unsatisfied
T1002,M1184,A107,2024-03-13T03:53:00.000+0000,2024-03-13T04:47:00.000+0000,Email,Low,Closed,54.0,1,Agent_7,East,Texas,2020-09-22,45,Non-binary,IL,2023-07-07,2,Excellent support
T1006,M1205,A102,2024-01-14T08:55:00.000+0000,2024-01-14T11:02:00.000+0000,Phone,Medium,Closed,127.0,0,Agent_2,East,Texas,2019-06-30,33,Male,FL,2023-09-08,null,null
T1005,M1174,A114,2024-01-04T18:18:00.000+0000,2024-01-04T20:23:00.000+0000,Email,Low,Closed,125.0,0,Agent_14,Central,Florida,2022-06-14,60,Male,WA,2023-06-07,null,null
T1001,M1268,A108,2024-02-17T06:13:00.000+0000,2024-02-17T09:27:00.000+0000,Email,High,Closed,194.0,0,Agent_8,East,New York,2020-12-21,50,Non-binary,TX,2024-03-15,5,Helpful
T1003,M1023,A108,2024-03-18T00:45:00.000+0000,2024-03-18T03:06:00.000+0000,Phone,Low,Closed,141.0,0,Agent_8,East,New York,2020-12-21,53,Non-binary,TX,2022-03-11,4,Excellent support
T1009,M1230,A112,2024-06-01T19:14:00.000+0000,2024-06-01T22:37:00.000+0000,Chat,Medium,Closed,203.0,0,Agent_12,Central,California,2021-12-16,52,Non-binary,NY,2023-11-22,null,null
T1007,M1076,A104,2024-01-19T18:40:00.000+0000,2024-01-19T21:40:00.000+0000,Phone,High,Closed,180.0,0,Agent_4,Central,New York,2019-12-27,59,Male,CA,2022-08-17,1,Could be better
T1004,M1251,A124,2024-05-21T09:44:00.000+0000,2024-05-21T10:21:00.000+0000,Chat,Low,Closed,37.0,1,Agent_24,Central,California,2024-11-30,27,Male,NY,2024-01-24,3,Helpful
T1008,M1213,A124,2024-03-18T02:15:00.000+0000,2024-03-18T02:50:00.000+0000,Chat,High,Closed,35.0,1,Agent_24,Central,California,2024-11-30,58,Female,IL,2023-10-02,3,Excellent support


In [0]:
%sql
-- Weekly performance summary by agent
CREATE OR REPLACE TEMP VIEW agent_weekly_summary AS
SELECT
  agent_id,
  name AS agent_name,
  team,
  location,
  weekofyear(created_at) AS week_num,
  COUNT(ticket_id) AS total_tickets,
  AVG(resolution_time_min) AS avg_resolution_time,
  SUM(sla_met) AS sla_met_count,
  ROUND(100.0 * SUM(sla_met) / COUNT(ticket_id), 2) AS sla_compliance_pct,
  ROUND(AVG(csat_rating), 2) AS avg_csat
FROM dashboard_data
GROUP BY agent_id, agent_name, team, location, weekofyear(created_at);


In [0]:
%sql
SELECT * FROM agent_weekly_summary ORDER BY week_num, agent_id LIMIT 10;

agent_id,agent_name,team,location,week_num,total_tickets,avg_resolution_time,sla_met_count,sla_compliance_pct,avg_csat
A102,Agent_2,East,Texas,1,1,68.0,0,0.00,4.0
A110,Agent_10,Central,Florida,1,1,159.0,0,0.00,null
A111,Agent_11,East,Florida,1,1,64.0,0,0.00,null
A114,Agent_14,Central,Florida,1,2,180.5,0,0.00,2.0
A116,Agent_16,East,Texas,1,2,87.5,0,0.00,5.0
A117,Agent_17,West,New York,1,3,91.0,2,66.67,4.0
A120,Agent_20,West,California,1,1,126.0,0,0.00,1.0
A122,Agent_22,Central,New York,1,1,160.0,0,0.00,2.0
A123,Agent_23,West,New York,1,2,218.0,0,0.00,3.5
A124,Agent_24,Central,California,1,1,65.0,0,0.00,4.0


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS credit_union;
USE credit_union;

In [0]:
final_df.write.format("delta").mode("overwrite").saveAsTable("credit_union.dashboard_data")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW daily_metrics AS
SELECT
  CAST(created_at AS DATE) AS date,
  COUNT(*) AS total_tickets,
  ROUND(AVG(resolution_time_min), 2) AS avg_resolution_time,
  ROUND(SUM(sla_met) / COUNT(*) * 100, 2) AS sla_compliance_pct
FROM dashboard_data
GROUP BY CAST(created_at AS DATE)
ORDER BY date;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW team_csat_trends AS
SELECT
  team,
  weekofyear(created_at) AS week,
  COUNT(*) AS ticket_count,
  ROUND(AVG(csat_rating), 2) AS avg_csat
FROM dashboard_data
GROUP BY team, weekofyear(created_at)
ORDER BY team, week;


In [0]:
%sql
SELECT
  agent_id,
  COUNT(*) AS total_tickets,
  ROUND(AVG(resolution_time_min), 2) AS avg_resolution,
  ROUND(SUM(sla_met) * 100.0 / COUNT(*), 2) AS sla_compliance_pct,
  ROUND(AVG(csat_rating), 2) AS avg_csat
FROM dashboard_data
GROUP BY agent_id
ORDER BY total_tickets DESC


agent_id,total_tickets,avg_resolution,sla_compliance_pct,avg_csat
A114,35,119.71,28.57,3.14
A122,26,127.15,26.92,2.95
A109,24,116.71,29.17,3.05
A115,24,130.38,16.67,2.61
A102,23,161.22,8.70,3.72
A113,23,126.83,17.39,2.79
A104,23,129.83,17.39,2.4
A120,22,118.77,27.27,2.56
A117,22,122.18,27.27,3.11
A119,21,144.62,14.29,2.44


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT team, ROUND(AVG(sla_met) * 100, 2) AS sla_pct
FROM dashboard_data
GROUP BY team


team,sla_pct
Central,26.18
East,20.89
West,22.62


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT CAST(created_at AS DATE) AS day, COUNT(*) AS tickets
FROM dashboard_data
GROUP BY CAST(created_at AS DATE)
ORDER BY day


day,tickets
2024-01-01,2
2024-01-02,2
2024-01-03,5
2024-01-04,1
2024-01-06,4
2024-01-07,1
2024-01-08,2
2024-01-10,7
2024-01-12,2
2024-01-13,3


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT location, ROUND(AVG(csat_rating), 2) AS avg_csat
FROM dashboard_data
GROUP BY location


location,avg_csat
Texas,3.12
Florida,3.09
California,2.87
New York,3.07


Databricks visualization. Run in Databricks to view.